In [1]:
import numpy as np

from two_step_classification import *
print_version()

python:      3.10.11
sklearn:     1.2.2
tensorflow:  2.12.0
keras:       2.12.0
numpy:       1.23.5
pandas:      1.5.3


In [2]:
# load data
x_train: pd.DataFrame = pd.read_pickle("models/kdd99_features/x_train-drop_25_df.pkl")
x_test: pd.DataFrame = pd.read_pickle("models/kdd99_features/x_test-drop_25_df.pkl")
y_train: pd.Series = pd.read_pickle("models/kdd99_features/y_train_df.pkl")
y_test: pd.Series = pd.read_pickle("models/kdd99_features/y_test_df.pkl")
x_train_ae_35: pd.DataFrame = pd.read_pickle("models/kdd99_features/x_train-drop+ae_35_df&activation=relu&epochs=5&batch_size=32.pkl")
x_test_ae_35: pd.DataFrame = pd.read_pickle("models/kdd99_features/x_test-drop+ae_35_df&activation=relu&epochs=5&batch_size=32.pkl")

In [3]:
lr_binary_dir = 'models/logistic_regression_binary/'
lr_anomaly_dir = 'models/logistic_regression_anomaly/'
lgb_binary_dir = 'models/lightgbm/'


In [4]:
#
with open("models/logistic_regression_binary/kdd99-drop_25&penalty=l1&solver=liblinear&C=0.1.pkl", 'rb') as fp:
    model_1st: LogisticRegression = pickle.load(fp)
with open("models/logistic_regression_anomaly/kdd99-dropped_mapped&penalty=l1&solver=liblinear&C=1e-05.pkl", 'rb') as fp:
    model_2nd: LogisticRegression = pickle.load(fp)
y_pred_binary = classification_normal_and_anomaly(x_test, model_1st)
predicted_indexes = y_pred_binary[y_pred_binary == 1].index
x_anomalies: pd.DataFrame = x_test.loc[predicted_indexes]
y_pred_anomalies: pd.Series = classification_anomalies(x_anomalies, model_2nd)
print(y_pred_anomalies.value_counts())
y_pred_normal = y_pred_binary[y_pred_binary == 0].apply(lambda _: 1)
y_pred = pd.concat([y_pred_normal, y_pred_anomalies])

y_test_binary = y_test.apply(lambda x: 0 if x == 1 else 1)

cm_1st = confusion_matrix_df(y_test_binary.sort_index(), y_pred_binary.sort_index(), labels=['normal', 'anomaly'])

cm_2nd = confusion_matrix_df(y_test.sort_index(), y_pred.sort_index())


0    129863
dtype: int64


In [5]:
cm_1st

,normal,anomaly
true_normal,31666,436
true_anomaly,1498,129427


In [6]:
cm_2nd

,dos,normal,probe,r2l,u2r
true_dos,128431,750,0,0,0
true_normal,436,31666,0,0,0
true_probe,896,459,0,0,0
true_r2l,92,280,0,0,0
true_u2r,8,9,0,0,0


In [7]:
model_1st: lgb.Booster = lgb.Booster(model_file="models/lightgbm/lgb_dropped_binary_tuned_booster.model")
model_2nd: lgb.Booster = lgb.Booster(model_file="models/lightgbm/lgb_dropped_mapped_anomaly_tuned_booster.model")
y_pred_binary = classification_normal_and_anomaly(x_test, model_1st)
predicted_indexes = y_pred_binary[y_pred_binary == 1].index
x_anomalies = x_test.loc[predicted_indexes]
y_pred_anomalies: pd.Series = classification_anomalies(x_anomalies, model_2nd)
print(y_pred_anomalies.value_counts())
y_pred_normal = y_pred_binary[y_pred_binary == 0].apply(lambda _: 1)
y_pred = pd.concat([y_pred_normal, y_pred_anomalies])

y_test_binary = y_test.apply(lambda x: 0 if x == 1 else 1)

cm_1st = confusion_matrix_df(y_test_binary.sort_index(), y_pred_binary.sort_index(), labels=['normal', 'anomaly'])

cm_2nd = confusion_matrix_df(y_test.sort_index(), y_pred.sort_index())

0    129631
1       761
2       380
3       144
dtype: int64


In [8]:
pd.Series(np.argmax(model_2nd.predict(x_train), axis=1)).value_counts()

0    297823
1     15490
2     15043
3      2638
dtype: int64

In [19]:
model_1st: lgb.Booster = lgb.Booster(model_file="models/lightgbm/lgb_dropped_binary_tuned_booster.model")
model_2nd: lgb.Booster = lgb.Booster(model_file="models/lightgbm/lgb_dropped_mapped_anomaly_tuned_booster.model")
y_pred_binary = classification_normal_and_anomaly(x_test, model_1st)
predicted_indexes = y_pred_binary[y_pred_binary == 1].index
x_anomalies = x_test.loc[predicted_indexes]
y_pred_anomalies: pd.Series = classification_anomalies(x_anomalies, model_2nd)
print(y_pred_anomalies.value_counts())
y_pred_normal = y_pred_binary[y_pred_binary == 0].apply(lambda _: 1)
y_pred = pd.concat([y_pred_normal, y_pred_anomalies])

y_test_binary = y_test.apply(lambda x: 0 if x == 1 else 1)

cm_1st = confusion_matrix_df(y_test_binary.sort_index(), y_pred_binary.sort_index(), labels=['normal', 'anomaly'])

cm_2nd = confusion_matrix_df(y_test.sort_index(), y_pred.sort_index(), labels=correspondences_anomaly.keys())

0    129631
1       761
2       380
3       144
dtype: int64


ValueError: Shape of passed values is (5, 5), indices imply (4, 4)

In [17]:
cm_1st

,normal,anomaly
true_normal,32083,19
true_anomaly,28,130897


In [18]:
cm_2nd

,dos,normal,probe,r2l,u2r
true_dos,129166,7,5,3,0
true_normal,2,32092,6,2,0
true_probe,463,757,5,130,0
true_r2l,0,9,359,4,0
true_u2r,0,7,5,5,0


In [20]:
confusion_matrix(y_test.sort_index(), y_pred.sort_index())

array([[129166,      7,      5,      3,      0],
       [     2,  32092,      6,      2,      0],
       [   463,    757,      5,    130,      0],
       [     0,      9,    359,      4,      0],
       [     0,      7,      5,      5,      0]])